### Train BBox with segmentaiton

In [3]:
# "/home/reuben/Atom360/Learning/data/dataset/coco"

!python main.py --coco_path "/home/wenisch/Atom360/AI/Learning/data/dataset"  \
        --coco_panoptic_path "panoptic" \
        --dataset_file "coco_panoptic" \
        --output_dir "output/panoptic/box_model"

Not using distributed mode
git:
  sha: 186dfa32689bbb37b0343a279f93a3f01c4dfdd0, status: has uncommited changes, branch: feature/panoptic-json

Namespace(aux_loss=True, backbone='resnet50', batch_size=2, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path='panoptic', coco_path='/home/wenisch/Atom360/AI/Learning/data/dataset', dataset_file='coco_panoptic', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=300, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=0.0001, lr_backbone=1e-05, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_queries=100, num_workers=2, output_dir='output/panoptic/box_model', position_embedding='sine', pre_norm=False, remove_difficult=False, resume='', seed=42, set_cost_bbox=5, set_cost_class=1, set_cost_giou=2, start_epoch=0, weight_decay=0.0001, world_size=1)
coco number of classes is 250
number of params: 41

In [2]:
#     --resume "weights/detr-r50_no-class-head.pth" \
# --start_epoch 20 \
!python main.py --coco_path /path/to/coco  
        --coco_panoptic_path /path/to/coco_panoptic 
        --dataset_file coco_panoptic
        --output_dir /output/path/box_model

# !python main.py --masks
#         --epochs 25
#         --lr_drop 15 
#         --coco_path /path/to/coco  
#         --coco_panoptic_path /path/to/coco_panoptic  
#         --dataset_file coco_panoptic 
#         --frozen_weights /output/path/box_model/checkpoint.pth 
#         --output_dir /output/path/segm_model

IndentationError: unexpected indent (1636717915.py, line 4)

In [ ]:
from util.plot_utils import plot_logs

from pathlib import Path

log_directory = [Path('outputs/')]

In [ ]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest)

#### Get Pretrained Weights
uncomment for downloading weights

In [ ]:
# import torch
# checkpoint = torch.hub.load_state_dict_from_url(
#             url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
#             map_location='cpu',
#             check_hash=True)

# # Remove class weights
# del checkpoint["model"]["class_embed.weight"]
# del checkpoint["model"]["class_embed.bias"]
# # del checkpoint["model"]["query_embed.weight"]

# # Save
# torch.save(checkpoint,
#            'weights/detr-r50_no-class-head2.pth')

### Test the model

In [ ]:
!python test.py \
    --dataset_file "oral" \
    --data_path "/home/wenisch/Atom360/AI/Learning/data/dataset" \
    --resume "outputs/checkpoint.pth"

### Load the fine-tuned model

In [ ]:
import torch
import matplotlib.pyplot as plt
import torchvision.transforms as T

num_classes = 3
finetuned_classes = ['N/A', 'spall', 'rebar', 'crack']
# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
model = torch.hub.load('facebookresearch/detr',
                       'detr_resnet50',
                       pretrained=False,
                       num_classes=num_classes)

checkpoint = torch.load('outputs/checkpoint.pth',
                        map_location='cpu')

model.load_state_dict(checkpoint['model'],
                      strict=False)

model.eval();

In [ ]:
def plot_finetuned_results(pil_img, prob=None, boxes=None):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if prob is not None and boxes is not None:
      for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
          ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                    fill=False, color=c, linewidth=3))
          cl = p.argmax()
          text = f'{finetuned_classes[cl]}: {p[cl]:0.2f}'
          ax.text(xmin, ymin, text, fontsize=15,
                  bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()
  
# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)
    
def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b
def filter_bboxes_from_outputs(outputs,
                               threshold=0.7):
  
  # keep only predictions with confidence above threshold
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > threshold

  probas_to_keep = probas[keep]

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  
  return probas_to_keep, bboxes_scaled

In [ ]:
def run_worflow(my_image, my_model):
  # mean-std normalize the input image (batch-size: 1)
  img = transform(my_image).unsqueeze(0)

  # propagate through the model
  outputs = my_model(img)
  # print(outputs)
  for threshold in [0.9, 0.5, 0.1]:
    
    probas_to_keep, bboxes_scaled = filter_bboxes_from_outputs(outputs,
                                                              threshold=threshold)

    plot_finetuned_results(my_image,
                           probas_to_keep, 
                           bboxes_scaled)


### With a training image

In [ ]:
from PIL import Image

img_name = "/home/wenisch/Atom360/AI/Learning/data/dataset/images/00019.jpg"
im = Image.open(img_name)


run_worflow(im,
            model)

In [ ]:
from PIL import Image

img_name = "/home/wenisch/Atom360/AI/Learning/data/dataset/images/00018.jpg"
im = Image.open(img_name)

run_worflow(im,
            model)